<a href="https://colab.research.google.com/github/threegenie/churn_project/blob/main/telco_customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **이 데이터를 선택한 이유 ?**
캐글에서 다양한 데이터를 찾던 중, 'Telco Customer Churn'이라는 데이터셋을 보게 되었습니다. 

최근에 핸드폰을 사용하면서 요금제 관련 불편한 점 때문에 기존에 오래 사용하던 통신사가 아닌 다른 통신사의 요금제로 바꾼 경험이 있었기 때문에, 관심이 생기는 주제였습니다. 

사람들이 통신사를 바꾸게 될 때 단순히 불편해서만이 아닌, 보편적인 이유가 있을지 궁금해져서 여러 특성들을 통해 통신사 변경 여부를 예측하는 모델을 만들어 보려고 합니다.

### **특성 설명**

* Gender : 성별 

* SeniorCitizen : 고령 여부 

* Partner : 배우자 유무 

* Dependents : 부양가족 유무 

* Tenure : 직장 근속 개월수 

* PhoneService : 전화 서비스 여부 

* MultipleLines : 다중 회선 여부

* InternetService : 인터넷 공급자 
  *  DSL
  *  Fiber optic

* OnlineSecurity : 온라인 보안 서비스 이용 여부

* OnlineBackup : 온라인 백업 서비스 이용 여부

* DeviceProtection : 기기보호 서비스 이용 여부

* TechSupport : 기술지원 서비스 이용 여부 

* StreamingTV : TV채널 스트리밍 이용 여부

* StreamingMovies : 영화 스트리밍 이용 여부 

* Contract : 계약 기간 
  * Month-to-Month 
  * One year 
  * Two year

* PaperlessBilling : 종이 고지서 신청 여부

* PaymentMethod : 요금 지불 방법
  * Electronic check
  * Bank transfer (automatic)
  * Mailed check
  * Credit card (automatic)

* MonthlyCharges : 월별 요금 

* TotalCharges : 총 요금 

* Churn : 이용자 서비스 해지 여부

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Telco-Customer-Churn.csv')
df = df.drop('customerID',axis=1)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


예측할 특성은 Churn, 즉 서비스 이용자가 통신사를 변경했는지의 여부입니다. 그래서 분류 모델을 이용해 이 문제를 해결할 것입니다.

In [ ]:
def processing(df):
  #TotalCharges - 숫자형인데 object로 표시되는 데이터를 numeric화
  df['TotalCharges'] = df['TotalCharges'].str.replace('.','')
  df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

  #Contract - 문자로 되어있는 계약 기간을 바꾸기
  mapping = {'Month-to-month':0,'One year':1,'Two year':2}
  df['Contract'] = df['Contract'].replace(mapping)

  #MultipleLines - yes / no / no phone service를 숫자로 바꾸기
  mapping2 = {'No phone service':0,'No':1,'Yes':2}
  df['MultipleLines'] = df['MultipleLines'].replace(mapping2)

  #그 외 yes/no/no internet service를 항목으로 가지는 특성들 숫자로 바꾸기
  mapping3 = {'No internet service':0,'No':1,'Yes':2}
  no_internet_feature = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
  df[no_internet_feature] = df[no_internet_feature].replace(mapping3)

  #Chrun - yes/no를 숫자로 바꾸기
  mapping4 = {'No':0,'Yes':1}
  df['Churn'] = df['Churn'].replace(mapping4)

processing(df)

In [ ]:
def features(df):
  #배우자와 부양가족이 모두 있는지 여부 - Yes : 1, No : 0
  df['Family'] = np.where((df['Partner']==1) & (df['Dependents']==1),1,0)

  #온라인 보안, 백업 서비스를 모두 이용하는지 여부
  df['OnlineService'] = np.where((df['OnlineSecurity']==2) & (df['OnlineBackup']==2),1,0)

  #tv채널과 영화 스트리밍 서비스를 모두 이용하는지 여부
  df['StreamingService'] = np.where((df['StreamingTV']==2) & (df['StreamingMovies']==2),1,0)

features(df)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Family,OnlineService,StreamingService
0,Female,0,Yes,No,1,No,0,DSL,1,2,1,1,1,1,0,Yes,Electronic check,29.85,2985.0,0,0,0,0
1,Male,0,No,No,34,Yes,1,DSL,2,1,2,1,1,1,1,No,Mailed check,56.95,188950.0,0,0,0,0
2,Male,0,No,No,2,Yes,1,DSL,2,2,1,1,1,1,0,Yes,Mailed check,53.85,10815.0,1,0,1,0
3,Male,0,No,No,45,No,0,DSL,2,1,2,2,1,1,1,No,Bank transfer (automatic),42.30,184075.0,0,0,0,0
4,Female,0,No,No,2,Yes,1,Fiber optic,1,1,1,1,1,1,0,Yes,Electronic check,70.70,15165.0,1,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.1, random_state=2)
train, test = train_test_split(train, test_size=0.05, random_state=2)
train.shape, val.shape, test.shape

((6021, 23), (705, 23), (317, 23))

In [ ]:
target = 'Churn'
features = ['gender', 'SeniorCitizen', 'tenure','PhoneService', 'MultipleLines', 
            'InternetService', 'DeviceProtection', 'TechSupport', 'Contract',
            'PaperlessBilling', 'PaymentMethod','MonthlyCharges', 'TotalCharges',
            'Family','OnlineService','StreamingService']

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]